# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
csv_file = os.path.join("..", "WeatherPy", "Weather_Py_data.csv")

vacation_df = pd.read_csv(csv_file)
vacation_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Leshukonskoye,99,RU,1603168845,99,64.90,45.76,28.40,2.91
1,1,Mount Isa,12,AU,1603168814,14,-20.73,139.50,98.60,6.93
2,2,Rikitea,0,PF,1603168637,78,-23.12,-134.97,74.26,13.47
3,3,La Asuncion,0,VE,1603168845,63,11.03,-63.86,91.40,10.29
4,4,Vaini,40,TO,1603168845,73,-21.20,-175.20,75.20,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [12]:
# Use Lat and Lng as locations and humidity as weight 
locations = vacation_df[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store variable for weights
humidity = vacation_df["Humidity"].astype(float)

In [14]:
# Create a humidity heatmap layer
fig = gmaps.figure()

# Pass through locations, set weight for color gradients
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity= 100, point_radius= 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Narrow down to ideal weather conditions: 

# 60-75 degrees max temperature
ideal_df = vacation_df[(vacation_df["Max Temp"] >= 60) & (vacation_df["Max Temp"] <= 75)]

# Winds less than or = 20 mph
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 20]

# Few clouds
ideal_df = ideal_df[ideal_df['Cloudiness'] <=30]

# Humidity less than or = 55
ideal_df = ideal_df[ideal_df['Humidity'] <= 55]

# Drop rows with null values
ideal_df=ideal_df.dropna()

ideal_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
10,10,Saint George,1,US,1603168659,14,37.10,-113.58,69.80,1.99
34,34,Yinchuan,4,CN,1603168932,25,38.47,106.27,63.46,19.04
241,241,Ōzu,20,JP,1603168953,49,33.50,132.55,73.40,16.11
257,257,Lake Shasta,18,US,1603168955,41,40.68,-122.37,70.00,5.57
263,263,Minzhu,0,CN,1603168956,44,43.73,127.34,62.22,11.21
274,274,Kutum,5,SD,1603168695,22,14.20,24.67,72.43,7.81
280,280,Izumo,20,JP,1603168662,52,35.37,132.77,69.80,6.93
287,287,El Dorado,1,US,1603168958,30,38.77,-120.52,69.80,6.93
290,290,Kamaishi,20,JP,1603168959,37,39.27,141.88,66.20,2.24
297,297,Atascadero,1,US,1603168959,42,35.49,-120.67,68.00,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
# Store into hotel DataFrame
hotel_df = ideal_df

# Add Hotel Name column to DataFrame
hotel_df['Hotel Name'] = ""

# Delete first column- redundant
del hotel_df["Unnamed: 0"]

hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
10,Saint George,1,US,1603168659,14,37.10,-113.58,69.80,1.99,
34,Yinchuan,4,CN,1603168932,25,38.47,106.27,63.46,19.04,
241,Ōzu,20,JP,1603168953,49,33.50,132.55,73.40,16.11,
257,Lake Shasta,18,US,1603168955,41,40.68,-122.37,70.00,5.57,
263,Minzhu,0,CN,1603168956,44,43.73,127.34,62.22,11.21,


In [32]:
# Set parameters to search for hotels within 5000 meters.
# Hit the Google Places API for each city's coordinates.

for index, row in hotel_df.iterrows():
    try:
        
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }

        lat = row["Lat"]
        lng = row["Lng"]

        params["location"] = f"{lat}, {lng}"

        hotel_data = requests.get(url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
10,Saint George,1,US,1603168659,14,37.10,-113.58,69.80,1.99,SpringHill Suites by Marriott St. George Washi...
34,Yinchuan,4,CN,1603168932,25,38.47,106.27,63.46,19.04,JW Marriott Hotel Yinchuan
241,Ōzu,20,JP,1603168953,49,33.50,132.55,73.40,16.11,NIPPONIA HOTEL Ozu Castle Town
257,Lake Shasta,18,US,1603168955,41,40.68,-122.37,70.00,5.57,NaN
263,Minzhu,0,CN,1603168956,44,43.73,127.34,62.22,11.21,Yashetian Xiangjingpin Hotel


In [33]:
# View entire DataFrame
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
10,Saint George,1,US,1603168659,14,37.10,-113.58,69.80,1.99,SpringHill Suites by Marriott St. George Washi...
34,Yinchuan,4,CN,1603168932,25,38.47,106.27,63.46,19.04,JW Marriott Hotel Yinchuan
241,Ōzu,20,JP,1603168953,49,33.50,132.55,73.40,16.11,NIPPONIA HOTEL Ozu Castle Town
257,Lake Shasta,18,US,1603168955,41,40.68,-122.37,70.00,5.57,NaN
263,Minzhu,0,CN,1603168956,44,43.73,127.34,62.22,11.21,Yashetian Xiangjingpin Hotel
274,Kutum,5,SD,1603168695,22,14.20,24.67,72.43,7.81,NaN
280,Izumo,20,JP,1603168662,52,35.37,132.77,69.80,6.93,スーパーホテル出雲駅前
287,El Dorado,1,US,1603168958,30,38.77,-120.52,69.80,6.93,Sly Park Resort
290,Kamaishi,20,JP,1603168959,37,39.27,141.88,66.20,2.24,Hotel Folkloro Sanriku Kamaishi
297,Atascadero,1,US,1603168959,42,35.49,-120.67,68.00,4.70,Holiday Inn Express Hotel & Suites Atascadero


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Plot markers to layer on top of heatmap
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))